> **Warning**: The A2A protocol is still under development and evolving rapidly. These examples reflect the implementation in the solution's `A2AServer` project.

## References

- [A2A Protocol Specification](https://google.github.io/A2A/)
- [A2A Server Implementation](../src/SKCodeAssistent/A2AServer/Program.cs)
- [A2A Client Implementation](../src/SKCodeAssistent/SKCodeAssistent.Server/SCHOOL_SOLUTIONS/CodingAssistentSessions/CodingAssistentSession_CustomOrchestrationWithA2A.cs)


## Setup and Configuration

In [13]:
#r "nuget: Microsoft.SemanticKernel, 1.67.1"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel CreateKernel()
{
    var builder = Kernel.CreateBuilder();

    var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, embeddingEndpoint, embeddingApiKey) = Settings.LoadFromFile();

    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
    var kernel = builder.Build();

    return kernel;
}

var kernel = CreateKernel();

Installed Packages Microsoft.SemanticKernel, 1.67.1

## Install Required Packages

In [14]:
// Install Semantic Kernel and A2A packages
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.67.1"
#r "nuget: A2A, 0.3.3-preview"
#r "nuget: A2A.AspNetCore, 0.3.3-preview"
#r "nuget: Microsoft.SemanticKernel.Agents.A2A, 1.67.1-alpha"

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;  // Alias to avoid conflicts
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;
using System.Text.Json;

Console.WriteLine("Packages loaded successfully.");

Installed Packages A2A, 0.3.3-preview A2A.AspNetCore, 0.3.3-preview Microsoft.SemanticKernel.Agents.A2A, 1.67.1-alpha Microsoft.SemanticKernel.Agents.Core, 1.67.1

Packages loaded successfully.


## Understanding A2A Protocol Messages

The A2A protocol uses JSON-RPC 2.0 for message exchange. Let's examine the key message types:

### 1. Agent Discovery (Agent Card)

Agents expose their capabilities via a well-known endpoint:

```http
GET /.well-known/agent.json
```

Response:
```json
{
  "name": "Invoice Agent",
  "description": "Handles invoice-related queries",
  "version": "1.0",
  "capabilities": ["invoice_lookup", "invoice_validation"],
  "endpoints": {
    "message": "/"
  }
}
```

### 2. Sending Messages

```json
{
  "jsonrpc": "2.0",
  "id": "1",
  "method": "message/send",
  "params": {
    "id": "conv_123",
    "message": {
      "role": "user",
      "messageId": "msg_1",
      "parts": [{
        "kind": "text",
        "text": "Show me invoice for Contoso"
      }]
    }
  }
}
```

### 3. Receiving Responses

```json
{
  "jsonrpc": "2.0",
  "id": "1",
  "result": {
    "message": {
      "role": "assistant",
      "messageId": "msg_2",
      "parts": [{
        "kind": "text",
        "text": "Invoice ID: INV-001, Amount: $1500.00"
      }]
    }
  }
}
```

## A2A Implementation


### 1. Create A2A Server (Server-Side)

This code demonstrates how to host a Semantic Kernel agent as an A2A server, based on `src\SKCodeAssistent\A2AServer\Program.cs`.


```csharp
using A2A;
using A2A.AspNetCore;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.A2A;

var builder = WebApplication.CreateBuilder();

// Add services
builder.Services.AddKernel();
// ... Add ChatCompletion services ...

var app = builder.Build();

// 1. Create the Semantic Kernel Agent
var codingAgent = new ChatCompletionAgent()
{
    Kernel = kernel,
    Name = "DevAssistant",
    Instructions = "You are a helpful assistant for developers.",
};

// 2. Define the Agent Card (Metadata)
var agentCard = new AgentCard()
{
    Name = "A2ACodingAgent",
    Description = "Semantic Kernel-based developer assistant.",
    Version = "1.0.0",
    Capabilities = new AgentCapabilities { Streaming = false },
    Skills = [
        new AgentSkill {
            Id = "dev_assistant_sk",
            Name = "Semantic Kernel Developer Assistant",
            Description = "Handles comprehensive developer assistance."
        }
    ]
};

// 3. Create A2A Host Agent and Map Endpoints
var a2aHostAgent = new A2AHostAgent(codingAgent, agentCard);
var taskManager = a2aHostAgent.TaskManager!;

app.MapA2A(taskManager, "/");
app.MapWellKnownAgentCard(taskManager, "/");

await app.RunAsync();

```

### 2. Consume A2A Agent (Client-Side) - make sure you run the apphost first

This code demonstrates how to connect to and use a remote A2A agent, based on [..\src\SKCodeAssistent\SKCodeAssistent.Server\SCHOOL_SOLUTIONS\CodingAssistentSessions\CodingAssistentSession_CustomOrchestrationWithA2A.cs](..\src\SKCodeAssistent\SKCodeAssistent.Server\SCHOOL_SOLUTIONS\CodingAssistentSessions\CodingAssistentSession_CustomOrchestrationWithA2A.cs).


In [20]:
using System.Net.Http;
using A2A;
using Microsoft.SemanticKernel.Agents.A2A;

// 1. Setup Client
var httpClient = new HttpClient();
var agentUrl = new Uri("http://localhost:65366"); // URL of the A2A Server
var a2aClient = new A2AClient(agentUrl, httpClient);

// 2. Resolve Agent Card (Discovery)
var cardResolver = new A2ACardResolver(agentUrl, httpClient);
var agentCard = await cardResolver.GetAgentCardAsync();

// 3. Create Local Proxy for Remote Agent
// A2AAgent wraps the client and card to provide an Agent interface
var remoteAgent = new A2AAgent(a2aClient, agentCard);

// 4. Invoke the Remote Agent
await foreach (var message in remoteAgent.InvokeAsync("Help me create a C# console app."))
{
    Console.WriteLine(message.Message.Content);
}


 Creating a C# console application involves several steps. Here'thy a guide to help you create a simple C# console application:

1. **Set up your development environment:**
   - Install Visual Studio or an IDE of your choice that supports C# development.
   - Ensure you have the .NET SDK installed, which is required for C# development.

2. **Create a new C# Console Application:**
   - Open Visual Studio or your IDE.
   - Go to `File` > `New` > `Project`.
   - Choose `Console App (.NET Core)` or `Console App (Windows Forms)` depending on your target platform.
   - Name your project, for example, `SimpleConsoleApp`.

3. **Write your C# code:**
   - In the `Program.cs` file, write your C# code. Here's a simple example:

```c



## Notes 

* **The .NET A2A libarary doesnt allow messages with role other than User to be sent to the A2A agent**

 You can use this wrapper to protect from this  
 ```csharp
 public sealed class A2ARemoteAgent : Agent
{    
    private readonly A2AAgent _a2AAgent;

    public A2ARemoteAgent(A2AAgent a2AAgent)
    {        
        _a2AAgent = a2AAgent;
        this.Name = a2AAgent.Name;
        this.Description = a2AAgent.Description;
    }

    public override IAsyncEnumerable<AgentResponseItem<ChatMessageContent>> InvokeAsync(ICollection<ChatMessageContent> messages, 
      Microsoft.SemanticKernel.Agents.AgentThread? thread = null, 
      AgentInvokeOptions? options = null, 
      CancellationToken cancellationToken = default)
    {
        StringBuilder fullMessage = new StringBuilder();
        foreach(var msg in messages)
        {
            fullMessage.AppendLine($"Role: {msg.Role} Content:{msg.Content}");
        }

        return _a2AAgent.InvokeAsync(new ChatMessageContent(AuthorRole.User, fullMessage.ToString()), thread, options).Select(m => { m.Message.AuthorName = Name; return m; });
        
    }

    public override IAsyncEnumerable<AgentResponseItem<StreamingChatMessageContent>> InvokeStreamingAsync(ICollection<ChatMessageContent> messages, 
      Microsoft.SemanticKernel.Agents.AgentThread? thread = null, 
      AgentInvokeOptions? options = null, 
      CancellationToken cancellationToken = default)
    {
        StringBuilder fullMessage = new StringBuilder();
        foreach (var msg in messages)
        {
            fullMessage.AppendLine($"Role: {msg.Role} Content:{msg.Content}");
        }
        return _a2AAgent.InvokeStreamingAsync(new ChatMessageContent(AuthorRole.User, fullMessage.ToString()), thread, options).Select(m => { m.Message.AuthorName = Name; return m; });       
    }

    protected override Task<AgentChannel> CreateChannelAsync(CancellationToken cancellationToken)
    {
        throw new NotSupportedException("A2AAgent is not for use with AgentChat.");
    }

    protected override IEnumerable<string> GetChannelKeys()
    {
        throw new NotSupportedException("A2AAgent is not for use with AgentChat.");
    }

    protected override Task<AgentChannel> RestoreChannelAsync(string channelState, CancellationToken cancellationToken)
    {
        throw new NotSupportedException("A2AAgent is not for use with AgentChat.");
    }
}
 ```

## Next Steps

- Explore the [A2A specification](https://google.github.io/A2A/)
- Try the [Semantic Kernel A2A samples](https://github.com/microsoft/semantic-kernel/tree/main/dotnet/samples/Demos/A2AClientServer)
- Build your own A2A server with SharpA2A.Core
- Test with the A2A Inspector tool
- See Assignment 4 for hands-on practice